In [ ]:
# ---------- CV runner: Stratified K-fold + repeated seeds ----------
def run_stratified_cv(
    X, y,
    num_classes: int,
    seeds=(42, 7, 2024),     # You can set to (42,) for faster run
    n_splits=5,
    epochs=200,
    batch_size=8,
    verbose_fit=0
):
    records = []

    for seed in seeds:
        set_all_seeds(seed)

        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

        for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), start=1):
            # graph between folds
            K.clear_session()
            set_all_seeds(seed)

            X_tr, X_te = X[train_idx], X[test_idx]
            y_tr, y_te = y[train_idx], y[test_idx]

            # ---- Leakage-free scaling----
            scaler = StandardScaler()
            X_tr = scaler.fit_transform(X_tr)
            X_te = scaler.transform(X_te)

            # ---- Reshape for CNN/LSTM branches ----
            X_tr_r = X_tr.reshape(X_tr.shape[0], X_tr.shape[1], 1)
            X_te_r = X_te.reshape(X_te.shape[0], X_te.shape[1], 1)

            # ---- Build and train model ----
            model = build_emhsam_cln(n_features=X_tr.shape[1], num_classes=num_classes)

            callbacks = [
                LearningRateScheduler(lr_schedule),

                EarlyStopping(monitor='val_accuracy', patience=25, restore_best_weights=True)
            ]

            model.fit(
                [X_tr_r, X_tr_r], y_tr,
                validation_data=([X_te_r, X_te_r], y_te),
                epochs=epochs,
                batch_size=batch_size,
                verbose=verbose_fit,
                callbacks=callbacks
            )

            # ---- Predict probabilities and labels ----
            prob = model.predict([X_te_r, X_te_r], verbose=0)
            pred = np.argmax(prob, axis=1)

            # ---- Metrics ----
            acc = accuracy_score(y_te, pred)
            macro_p = precision_score(y_te, pred, average='macro', zero_division=0)
            macro_r = recall_score(y_te, pred, average='macro', zero_division=0)
            macro_f1 = f1_score(y_te, pred, average='macro', zero_division=0)
            mcc = matthews_corrcoef(y_te, pred)
            kappa = cohen_kappa_score(y_te, pred)

            # Macro AUC (one-vs-rest)
            try:
                auc_macro = roc_auc_score(y_te, prob, multi_class='ovr', average='macro')
            except ValueError:
                auc_macro = np.nan

            records.append({
                "seed": seed,
                "fold": fold,
                "acc": acc,
                "macro_precision": macro_p,
                "macro_recall": macro_r,
                "macro_f1": macro_f1,
                "mcc": mcc,
                "kappa": kappa,
                "auc_macro_ovr": auc_macro
            })

            print(f"Seed={seed} Fold={fold} | "
                  f"Acc={acc:.4f} MacroF1={macro_f1:.4f} MCC={mcc:.4f} Kappa={kappa:.4f} AUC={auc_macro:.4f}")

    return pd.DataFrame.from_records(records)


# ---------- 11) Run CV ----------
# Adjust seeds if runtime is too high:
#  - seeds=(42,) gives 5 runs total (5 folds)
#  - seeds=(42,7,2024) gives 15 runs total (5 folds × 3 seeds)
results_df = run_stratified_cv(
    X_all, y_all,
    num_classes=num_classes,
    seeds=(42, 7, 2024),
    n_splits=5,
    epochs=200,
    batch_size=8,
    verbose_fit=0
)

print("\n===== Fold/Seed Results =====")
print(results_df)


# ---------- 12) Mean ± Std summary ----------
metrics = ["acc", "macro_precision", "macro_recall", "macro_f1", "mcc", "kappa", "auc_macro_ovr"]

summary = results_df[metrics].agg(['mean', 'std']).T
summary["mean±std"] = summary.apply(lambda r: f"{r['mean']:.4f} ± {r['std']:.4f}", axis=1)

print("\n===== Mean ± Std (Across all folds and seeds) =====")
print(summary[["mean±std"]])

# save for result:
results_df.to_csv("emhsam_cln_cv_all_runs.csv", index=False)
summary.to_csv("emhsam_cln_cv_summary_mean_std.csv")
print("\nSaved: emhsam_cln_cv_all_runs.csv and emhsam_cln_cv_summary_mean_std.csv")
